In [1]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO 
from pyAudioAnalysis import MidTermFeatures as mF
import numpy as np 
import plotly.graph_objs as go 
import plotly
import IPython

import pandas as pd

import os, shutil
import time

import matplotlib.pyplot as plt

import random



/opt/anaconda3/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## OnlyCorrect

In [53]:
df_train = pd.read_csv('../Data/Mid_features/midFeaturesTrainFinalWithChars.csv')
df_test = pd.read_csv('../Data/Mid_features/midFeaturesTestFinalWithChars.csv')


In [55]:
df_train.sort_values(by=['FileID']);
df_test.sort_values(by=['FileID']);

In [21]:
summary = pd.read_csv('../Data/CREMADSummaryTable.csv')
summary

,Unnamed: 0,FileName,VoiceVote,VoiceLevel,FaceVote,FaceLevel,MultiModalVote,MultiModalLevel
0,1,1001_IEO_NEU_XX,N,69.1,N,92.22,N,64.78
1,2,1001_IEO_HAP_LO,N,71.67,H,57,H,57.38
2,3,1001_IEO_HAP_MD,N,67.71,H,62.62,H,56.56
3,4,1001_IEO_HAP_HI,H,63.5,H,68.25,H,73.2
4,5,1001_IEO_SAD_LO,N,73.71,N,73.5,N,74.8
...,...,...,...,...,...,...,...,...
7437,7438,1091_WSI_HAP_XX,N,68.12,H,65.12,H,64.8
7438,7439,1091_WSI_SAD_XX,N,55.67,N,71.57,N,48.8
7439,7440,1091_WSI_ANG_XX,A,34.4,A,50.43,A,67.11
7440,7441,1091_WSI_FEA_XX,S,39.67,F,68.29,F,51.71


In [24]:
summary.drop('Unnamed: 0', inplace = True, axis = 1)

### Example trial for Logical Comparison

In [29]:
summary['FileName'][0][9]==summary['VoiceVote'][0]

True

In [28]:
summary['VoiceVote'][0]

'N'

## Calculating the "correctness"

In [109]:
# Creating a new column and saving the real label code as a single letter

summary['filecode'] = np.nan
for i in np.arange(len(summary['FileName'])):
    summary['filecode'][i] = summary['FileName'][i][9];

<ipython-input-109-9c977deaf49b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['filecode'][i] = summary['FileName'][i][9];


In [66]:
# Creating a new column and saving whether participants responded correctly to those in the "filecode"

summary['is_correct'] = np.nan
for i in np.arange(len(summary['FileName'])):
    x = (summary['filecode'][i]==summary['VoiceVote'][i])
    if x == True:
        summary['is_correct'][i] = 1
    elif x == False:
        summary['is_correct'][i] = 0;

<ipython-input-66-c1bf398b8e81>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['is_correct'][i] = 1
<ipython-input-66-c1bf398b8e81>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['is_correct'][i] = 0;


In [71]:
summary

,FileName,VoiceVote,VoiceLevel,FaceVote,FaceLevel,MultiModalVote,MultiModalLevel,is_correct,filecode
0,1001_IEO_NEU_XX,N,69.1,N,92.22,N,64.78,1.0,N
1,1001_IEO_HAP_LO,N,71.67,H,57,H,57.38,0.0,H
2,1001_IEO_HAP_MD,N,67.71,H,62.62,H,56.56,0.0,H
3,1001_IEO_HAP_HI,H,63.5,H,68.25,H,73.2,1.0,H
4,1001_IEO_SAD_LO,N,73.71,N,73.5,N,74.8,0.0,S
...,...,...,...,...,...,...,...,...,...
7437,1091_WSI_HAP_XX,N,68.12,H,65.12,H,64.8,0.0,H
7438,1091_WSI_SAD_XX,N,55.67,N,71.57,N,48.8,0.0,S
7439,1091_WSI_ANG_XX,A,34.4,A,50.43,A,67.11,1.0,A
7440,1091_WSI_FEA_XX,S,39.67,F,68.29,F,51.71,0.0,F


In [52]:
# Save them all into csv

summary.to_csv('../Data/Mid_features/summary_corrLabel.csv', index=False)

## Getting only train set values from the summary table

In [58]:
df_train.insert(loc = 4,
              column = 'is_correct',
              value = np.nan)

ValueError: cannot insert is_correct, already exists

In [60]:
df_test.insert(loc = 4,
              column = 'is_correct',
              value = np.nan)


In [76]:
df_train.head()

,FileID,actorID,Emotion,SentenceID,is_correct,Age,Sex,Race,Ethnicity,zcr_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1001_DFA_ANG_XX,1001,ANG,DFA,NaN,51,Male,Caucasian,Not Hispanic,0.159956,...,0.024107,0.014803,0.017961,0.013412,0.008655,0.010352,0.009738,0.010600,0.004328,0.009167
1,1001_DFA_DIS_XX,1001,DIS,DFA,NaN,51,Male,Caucasian,Not Hispanic,0.175069,...,0.022395,0.015510,0.008768,0.014533,0.009661,0.002533,0.004223,0.007513,0.003662,0.007296
2,1001_DFA_FEA_XX,1001,FEA,DFA,NaN,51,Male,Caucasian,Not Hispanic,0.199849,...,0.007043,0.003129,0.006915,0.007791,0.013899,0.005247,0.003474,0.014306,0.005781,0.008250
3,1001_DFA_HAP_XX,1001,HAP,DFA,NaN,51,Male,Caucasian,Not Hispanic,0.148663,...,0.021737,0.005675,0.009277,0.026797,0.010147,0.010658,0.017229,0.013203,0.010011,0.007488
4,1001_DFA_NEU_XX,1001,NEU,DFA,NaN,51,Male,Caucasian,Not Hispanic,0.174283,...,0.031970,0.012929,0.017969,0.037496,0.013379,0.008354,0.005615,0.008907,0.007483,0.013592


In [97]:
for i in np.arange(len(df_train['FileID'])):    
    x = df_train['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_train['is_correct'][i] = y['is_correct']

<ipython-input-97-6a8e50c10df0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['is_correct'][i] = y['is_correct']


In [99]:
for i in np.arange(len(df_test['FileID'])):    
    x = df_test['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_test['is_correct'][i] = y['is_correct']

<ipython-input-99-69a25508e7eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['is_correct'][i] = y['is_correct']


In [100]:
df_train.head()

,FileID,actorID,Emotion,SentenceID,is_correct,Age,Sex,Race,Ethnicity,zcr_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1001_DFA_ANG_XX,1001,ANG,DFA,1.0,51,Male,Caucasian,Not Hispanic,0.159956,...,0.024107,0.014803,0.017961,0.013412,0.008655,0.010352,0.009738,0.010600,0.004328,0.009167
1,1001_DFA_DIS_XX,1001,DIS,DFA,1.0,51,Male,Caucasian,Not Hispanic,0.175069,...,0.022395,0.015510,0.008768,0.014533,0.009661,0.002533,0.004223,0.007513,0.003662,0.007296
2,1001_DFA_FEA_XX,1001,FEA,DFA,1.0,51,Male,Caucasian,Not Hispanic,0.199849,...,0.007043,0.003129,0.006915,0.007791,0.013899,0.005247,0.003474,0.014306,0.005781,0.008250
3,1001_DFA_HAP_XX,1001,HAP,DFA,0.0,51,Male,Caucasian,Not Hispanic,0.148663,...,0.021737,0.005675,0.009277,0.026797,0.010147,0.010658,0.017229,0.013203,0.010011,0.007488
4,1001_DFA_NEU_XX,1001,NEU,DFA,1.0,51,Male,Caucasian,Not Hispanic,0.174283,...,0.031970,0.012929,0.017969,0.037496,0.013379,0.008354,0.005615,0.008907,0.007483,0.013592


In [101]:
df_test.head()

,FileID,actorID,Emotion,SentenceID,is_correct,Age,Sex,Race,Ethnicity,zcr_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1006_DFA_ANG_XX,1006,ANG,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.205303,...,0.009290,0.011798,0.012141,0.005553,0.001191,0.005477,0.013535,0.017165,0.008230,0.008214
1,1006_DFA_DIS_XX,1006,DIS,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.142912,...,0.043360,0.034025,0.037411,0.016621,0.002165,0.006136,0.005255,0.008078,0.006201,0.013113
2,1006_DFA_FEA_XX,1006,FEA,DFA,0.0,58,Female,Caucasian,Not Hispanic,0.150361,...,0.016754,0.011521,0.040531,0.011855,0.010751,0.012277,0.005750,0.014204,0.013292,0.012245
3,1006_DFA_HAP_XX,1006,HAP,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.117750,...,0.033142,0.021647,0.012535,0.028002,0.018135,0.012021,0.022196,0.020446,0.008418,0.012261
4,1006_DFA_NEU_XX,1006,NEU,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.118134,...,0.026941,0.010169,0.038335,0.015219,0.011879,0.015302,0.008216,0.013460,0.016431,0.012039


In [102]:
df_train.to_csv('../Data/Mid_features/df_train_corrLabel.csv', index=False)

In [108]:
df_test.to_csv('../Data/Mid_features/df_test_all.csv', index=False)

In [105]:
df_test_corrlabel = df_test[df_test['is_correct'] == 1]

In [106]:
df_test_corrlabel.to_csv

,FileID,actorID,Emotion,SentenceID,is_correct,Age,Sex,Race,Ethnicity,zcr_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1006_DFA_ANG_XX,1006,ANG,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.205303,...,0.009290,0.011798,0.012141,0.005553,0.001191,0.005477,0.013535,0.017165,0.008230,0.008214
1,1006_DFA_DIS_XX,1006,DIS,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.142912,...,0.043360,0.034025,0.037411,0.016621,0.002165,0.006136,0.005255,0.008078,0.006201,0.013113
3,1006_DFA_HAP_XX,1006,HAP,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.117750,...,0.033142,0.021647,0.012535,0.028002,0.018135,0.012021,0.022196,0.020446,0.008418,0.012261
4,1006_DFA_NEU_XX,1006,NEU,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.118134,...,0.026941,0.010169,0.038335,0.015219,0.011879,0.015302,0.008216,0.013460,0.016431,0.012039
5,1006_DFA_SAD_XX,1006,SAD,DFA,1.0,58,Female,Caucasian,Not Hispanic,0.115795,...,0.022490,0.013506,0.030509,0.026745,0.006870,0.014035,0.009864,0.018840,0.012973,0.013929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,1089_TSI_NEU_XX,1089,NEU,TSI,1.0,24,Female,Caucasian,Not Hispanic,0.301379,...,0.028692,0.024063,0.006848,0.013698,0.002913,0.013846,0.009565,0.003859,0.003258,0.009791
1194,1089_WSI_ANG_XX,1089,ANG,WSI,1.0,24,Female,Caucasian,Not Hispanic,0.229983,...,0.002990,0.001033,0.004155,0.003678,0.001318,0.019570,0.003408,0.002347,0.026694,0.009494
1195,1089_WSI_DIS_XX,1089,DIS,WSI,1.0,24,Female,Caucasian,Not Hispanic,0.383122,...,0.004918,0.013561,0.010522,0.009827,0.005967,0.010294,0.010915,0.004520,0.010917,0.005369
1198,1089_WSI_NEU_XX,1089,NEU,WSI,1.0,24,Female,Caucasian,Not Hispanic,0.139094,...,0.023018,0.010559,0.011100,0.021006,0.007919,0.003927,0.009988,0.009861,0.020112,0.007493


In [107]:
df_test_corrlabel.to_csv('../Data/Mid_features/df_test_only_corrLabel.csv', index=False)